In [68]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [2]:
data = pd.read_csv('/Users/sazid/Documents/vinlus dataset/GPS Spoofing Detection with Parallel Computing/aisdk-2006-03/aisdk_20060302.csv')

In [3]:
data1 = data.rename(columns = {'# Timestamp':'Timestamp'})
data1['Timestamp']=pd.to_datetime(data1['Timestamp'], format='%d/%m/%Y %H:%M:%S')
data1['ETA']= pd.to_datetime(data1['ETA'], format='%d/%m/%Y %H:%M:%S', errors='coerce')




In [ ]:

with pd.option_context('display.max_rows', None,'display.max_columns', None):
    display(data1[
    
    data1['ROT'].isnull() &
    data1['SOG'].isnull() &
    data1['Navigational status'].str.contains('Unknown value',case=False, na=False)
    
    ])

In [4]:
data1 = data1[ ~(
    
    data1['ROT'].isnull() &
    data1['SOG'].isnull() &
    data1['Navigational status'].str.contains('Unknown value',case=False, na=False)
    
)] 

In [5]:
data1 = data1.drop(['IMO','Callsign','Cargo type','Ship type'], axis=1)

In [51]:
data1['SOG'].isnull().sum()

2532

In [52]:
data1['SOG'] = data1['SOG'].fillna(0)

In [127]:
data1 =  data1[((data1['Latitude']<=90) & (data1['Latitude']>= -90)) & ((data1['Longitude']<=180 ) & (data1['Longitude']>= -180))]

In [128]:
data1

,Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,Name,Width,Length,Type of position fixing device,Draught,Destination,ETA,Data source type
336,2006-03-02 00:00:46,Class A,0,56.039903,12.689080,Unknown value,NaN,0.0,0.0,NaN,JOTRON0,NaN,NaN,GPS,NaN,NaN,NaT,AIS
2750,2006-03-02 00:05:56,Class A,0,56.039903,12.689080,Unknown value,NaN,0.0,0.0,NaN,JOTRON0,NaN,NaN,GPS,NaN,NaN,NaT,AIS
2867,2006-03-02 00:06:11,Class A,0,56.039903,12.689080,Unknown value,NaN,0.0,0.0,NaN,JOTRON0,NaN,NaN,GPS,NaN,NaN,NaT,AIS
3153,2006-03-02 00:06:47,Class A,0,56.039903,12.689080,Unknown value,NaN,0.0,0.0,NaN,JOTRON0,NaN,NaN,GPS,NaN,NaN,NaT,AIS
5674,2006-03-02 00:12:15,Class A,0,56.039903,12.689080,Unknown value,NaN,0.0,0.0,NaN,JOTRON0,NaN,NaN,GPS,NaN,NaN,NaT,AIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374726,2006-03-02 12:30:55,Class A,999999999,55.475503,8.429478,Unknown value,NaN,0.0,141.2,NaN,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
374894,2006-03-02 12:31:05,Class A,999999999,55.475508,8.429477,Unknown value,NaN,0.1,175.4,NaN,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
375056,2006-03-02 12:31:15,Class A,999999999,55.475515,8.429472,Unknown value,NaN,0.4,162.7,NaN,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
375220,2006-03-02 12:31:25,Class A,999999999,55.475528,8.429468,Unknown value,NaN,0.0,145.4,NaN,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS


In [129]:
data1 = data1.sort_values(by=['MMSI','Timestamp'])

In [130]:
groups = tuple(data1.groupby('MMSI'))

In [131]:
len(groups)

2258

In [132]:
chunk_size = 10

In [133]:
chunk = [groups[i:i+chunk_size] for i in range(0, len(groups), chunk_size)]

In [134]:
len(chunk)

226

In [255]:
container = chunk[224]
mmsi , data2 = container[9]

In [258]:
time2 = None
lat1 = None
lon1 = None
count = 0
flag_count = 0
final_flag_count = 0
jump_count = 0
flag_sequence = 0
flag_sequence_count = 0

for i,row in data2.iterrows():
    count+=1
    
    lat = row['Latitude']
    lon = row['Longitude']
    point = (lat,lon)   
    time1 = row['Timestamp']
    speed = row['SOG'] * 1.852

    if count>=2:
        time_dif = (time1 - time2).total_seconds()/60           #in minutes
        dis = geodesic(point, point1).kilometers 
        max_can_go = 1 * time_dif                               #as at 30 knot(.92 km/m) speed, ship can go 1km max in 1 minutes
        print('Distance: ', dis,'KM','  Time difference: ',time_dif, 'maximum distance can cover in this time: ', max_can_go, 'Actual speed: ', 
              speed, 
              'with actual speed should go: ', speed/60 * time_dif, time1)


        #detecting mid level anamolies 
        if   (dis > max_can_go*3) and (dis<max_can_go * 50)  :
            
            
            flag_count+=1
            ob = data.index[count]-1
            print(ob)
            
            if (flag_sequence == ob) :
                flag_sequence_count += 1
                print('scon',flag_sequence_count)
                
            if (flag_sequence != ob) :
                 flag_sequence_count = 0

                    
            if flag_sequence_count >=2:
                final_flag_count+=1
                flag_sequence_count = 0
           

            print(flag_sequence)
            flag_sequence = data.index[count]
         
            print('flag  ')

            #detecting jump (serious)

        if dis > max_can_go * 50 :
            jump_count+=1
            print('Mega Jump  ')
                  
   
    time2 = time1
    lat1 = lat
    lon1 = lon
    point1 = (lat1,lon1)


print('Total flag: ', flag_count,'  Mega Jump: ',jump_count, count) 
print('final_flag: ', final_flag_count)

Distance:  0.0 KM   Time difference:  2.2666666666666666 maximum distance can cover in this time:  2.2666666666666666 Actual speed:  0.0 with actual speed should go:  0.0 2006-03-02 00:11:09
Distance:  0.0018925082271899925 KM   Time difference:  3.75 maximum distance can cover in this time:  3.75 Actual speed:  0.0 with actual speed should go:  0.0 2006-03-02 00:14:54
Distance:  0.0 KM   Time difference:  8.25 maximum distance can cover in this time:  8.25 Actual speed:  0.0 with actual speed should go:  0.0 2006-03-02 00:23:09
Distance:  0.0038311876534665393 KM   Time difference:  0.7166666666666667 maximum distance can cover in this time:  0.7166666666666667 Actual speed:  0.0 with actual speed should go:  0.0 2006-03-02 00:23:52
Distance:  0.0 KM   Time difference:  5.283333333333333 maximum distance can cover in this time:  5.283333333333333 Actual speed:  0.0 with actual speed should go:  0.0 2006-03-02 00:29:09
Distance:  0.001892508217378593 KM   Time difference:  0.7666666666

In [12]:
with pd.option_context('display.max_rows', None,'display.max_columns', None):
 display(groups[2262])

(999999999,
                  Timestamp Type of mobile       MMSI   Latitude  Longitude  \
 342367 2006-03-02 11:42:06        Class A  999999999  55.475705   8.428897   
 342482 2006-03-02 11:42:16        Class A  999999999  55.475698   8.428930   
 342706 2006-03-02 11:42:26        Class A  999999999  55.475693   8.428955   
 342967 2006-03-02 11:42:36        Class A  999999999  55.475692   8.428973   
 343153 2006-03-02 11:42:47        Class A  999999999  55.475688   8.428982   
 343296 2006-03-02 11:42:57        Class A  999999999  55.475687   8.428988   
 343411 2006-03-02 11:43:07        Class A  999999999  55.475687   8.428987   
 343515 2006-03-02 11:43:17        Class A  999999999  55.475687   8.428992   
 343603 2006-03-02 11:43:27        Class A  999999999  55.475687   8.428993   
 343696 2006-03-02 11:43:37        Class A  999999999  55.475687   8.428992   
 343791 2006-03-02 11:43:47        Class A  999999999  55.475687   8.428985   
 343886 2006-03-02 11:43:57        Class

In [79]:
with pd.option_context('display.max_columns',None, 'display.max_rows',None):
 display(data1[data1['MMSI']==1193046])

,Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,Name,Width,Length,Type of position fixing device,Draught,Destination,ETA,Data source type
228,2006-03-02 00:00:33,Class A,1193046,54.855108,10.522455,Under way using engine,NaN,0.0,0.0,NaN,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
1762,2006-03-02 00:03:53,Class A,1193046,57.688433,11.855517,Moored,0.0,0.0,111.0,271.0,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
2822,2006-03-02 00:06:05,Class A,1193046,57.688433,11.855517,Moored,0.0,0.0,111.0,271.0,NAUTICAST,43.0,220.0,GPS,2.0,NaN,NaT,AIS
3022,2006-03-02 00:06:33,Class A,1193046,54.855108,10.522455,Under way using engine,NaN,0.0,0.0,NaN,NAUTICAST,43.0,220.0,GPS,2.0,NaN,NaT,AIS
5817,2006-03-02 00:12:33,Class A,1193046,54.855108,10.522455,Under way using engine,NaN,0.0,0.0,NaN,NAUTICAST,43.0,220.0,GPS,2.0,NaN,NaT,AIS
5959,2006-03-02 00:12:52,Class A,1193046,54.855108,10.522455,Under way using engine,NaN,0.0,0.0,NaN,MSC PATRICIA,43.0,355.0,GPS,7.5,GOTEBORG,2006-03-01 16:15:00,AIS
5963,2006-03-02 00:12:53,Class A,1193046,57.688433,11.855567,Moored,0.0,0.0,111.0,271.0,MSC PATRICIA,43.0,355.0,GPS,7.5,GOTEBORG,2006-03-01 16:15:00,AIS
8674,2006-03-02 00:18:41,Class A,1193046,54.855108,10.522455,Under way using engine,NaN,0.0,0.0,NaN,MSC PATRICIA,43.0,355.0,GPS,7.5,GOTEBORG,2006-03-01 16:15:00,AIS
8765,2006-03-02 00:18:53,Class A,1193046,54.855108,10.522455,Under way using engine,NaN,0.0,0.0,NaN,MSC PATRICIA,43.0,355.0,GPS,7.5,GOTEBORG,2006-03-01 16:15:00,AIS
8770,2006-03-02 00:18:54,Class A,1193046,57.688417,11.855567,Moored,0.0,0.0,111.0,271.0,MSC PATRICIA,43.0,355.0,GPS,7.5,GOTEBORG,2006-03-01 16:15:00,AIS


In [2]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(user_agent="my_geocoder")

def get_location(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        return location.address
    except GeocoderTimedOut:
        return "Timeout"
    except:
        return "Error"

# Example
lat = 23.871831
lon = 90.319903
print(get_location(lat, lon))

আশুলিয়া মডেল টাউন, দত্তপাড়া, সাভার উপজেলা, ঢাকা জেলা, ঢাকা বিভাগ, 1341, বাংলাদেশ
